## MiBiG

In [1]:
import os
import glob
from Bio import SeqIO
import pandas as pd

In [2]:
def extract_module_info(gbk_file):
    module_data = []
    bgc_id = os.path.splitext(os.path.basename(gbk_file))[0]

    for record in SeqIO.parse(gbk_file, "genbank"):
        for feature in record.features:
            if feature.type == "aSModule":
                start, end = feature.location.start, feature.location.end
                length = end - start + 1
                module_type = feature.qualifiers.get("type", ["Unknown"])[0]
                
                module_data.append({
                    "BGC-id": bgc_id,
                    "Module Type": module_type,
                    "Length": length
                })

    return module_data

In [3]:
all_module_info = pd.DataFrame()

for file in glob.glob("./mibig_gbk_3.1/*.gbk"):
    module_info = pd.DataFrame(extract_module_info(file))
    all_module_info = pd.concat([all_module_info, module_info])

In [4]:
# Describe with respect to 'Module Type'
all_module_info[['Module Type', 'Length']].groupby("Module Type").describe()

Length                                                           \
              count         mean          std    min     25%     50%     75%   
Module Type                                                                    
nrps         3795.0  3290.751252   843.566828  733.0  3061.0  3127.0  3931.0   
pks          4781.0  4535.502196  1343.057480  685.0  3751.0  4591.0  5266.0   
unknown       261.0  1345.777778   502.704521  595.0   907.0  1195.0  1780.0   

                      
                 max  
Module Type           
nrps          6313.0  
pks          10133.0  
unknown       3181.0

## BiG-SLICE

In [5]:
import sqlite3
import pandas as pd
import numpy as np

In [6]:
# Get tables from the bigslice database into dataframes
with sqlite3.connect("./full_run_result/result/data.db") as con:
    cur = con.cursor()
    
    bgc_df = pd.read_sql_query(f"select * from bgc;", con)
    #print(bgc_df)
    
    bgc_class_df = pd.read_sql_query(f"select * from bgc_class;", con)
    #print(bgc_class_df)
    
    chem_class_df = pd.read_sql_query(f"select * from chem_class;", con)
    #print(chem_class_df)
    
    chem_subclass_df = pd.read_sql_query(f"select * from chem_subclass;", con)
    #print(chem_subclass_df)

In [7]:
# join chem_class and chem_subclass
chem_class_subclass_join = pd.merge(chem_class_df, chem_subclass_df, how="inner", left_on="id", right_on="class_id")

# rename columns
chem_class_subclass_join.rename(columns = {'id_x':'chem_class_id', 'name_x':'chem_class_name', 'id_y':'chem_subclass_id', 'name_y':'chem_subclass_name'}, inplace = True)

# subselect columns
chem_class_subclass_join = chem_class_subclass_join[['chem_class_id', 'chem_class_name', 'chem_subclass_id', 'chem_subclass_name']]
chem_class_subclass_join

,chem_class_id,chem_class_name,chem_subclass_id,chem_subclass_name
0,1,Unknown,1,unknown
1,2,Other,4,acyl_amino_acids
2,2,Other,5,arylpolyene
3,2,Other,6,fatty_acid
4,2,Other,7,other
...,...,...,...,...
117,8,Terpene,16,generic
118,8,Terpene,68,terpene
119,8,Terpene,114,unknown
120,8,Terpene,120,sesterterpene


In [8]:
# join bgc and bgc_class
bgc_join_bgc_class = pd.merge(bgc_df, bgc_class_df, how="inner", left_on="id", right_on="bgc_id")

# subselect columns
bgc_join_bgc_class = bgc_join_bgc_class[['bgc_id', 'dataset_id', 'name', 'type', 'on_contig_edge', 'length_nt','orig_folder', 'orig_filename', 'chem_subclass_id']]

# join chem_class_subclass_join and bgc_join_bgc_class
bgc_metadata = pd.merge(bgc_join_bgc_class, chem_class_subclass_join, how="inner", left_on="chem_subclass_id", right_on="chem_subclass_id")
bgc_metadata

,bgc_id,dataset_id,name,type,on_contig_edge,length_nt,orig_folder,orig_filename,chem_subclass_id,chem_class_id,chem_class_name,chem_subclass_name
0,1,1,BGC0001286.1,mibig,0,16181,,BGC0001286.gbk,110,2,Other,unknown
1,12,1,BGC0000830.1,mibig,0,8319,,BGC0000830.gbk,110,2,Other,unknown
2,15,1,BGC0000910.1,mibig,0,1420,,BGC0000910.gbk,110,2,Other,unknown
3,16,1,BGC0000907.1,mibig,0,38217,,BGC0000907.gbk,110,2,Other,unknown
4,27,1,BGC0000899.1,mibig,0,17665,,BGC0000899.gbk,110,2,Other,unknown
...,...,...,...,...,...,...,...,...,...,...,...,...
1415904,1186917,5,GCA_010211705.1/WJIA01000326.1.region001,as5,1,28836,GCA_010211705.1,WJIA01000326.1.region001.gbk,64,6,RiPP,fungal-RiPP
1415905,1186984,5,GCA_008729105.1/PPFZ01000005.1.region001,as5,0,46431,GCA_008729105.1,PPFZ01000005.1.region001.gbk,64,6,RiPP,fungal-RiPP
1415906,1187012,5,GCA_008729105.1/PPFZ01000007.1.region004,as5,0,44164,GCA_008729105.1,PPFZ01000007.1.region004.gbk,64,6,RiPP,fungal-RiPP
1415907,1188506,5,GCA_002975665.1/PORH01000028.1.region001,as5,0,53535,GCA_002975665.1,PORH01000028.1.region001.gbk,64,6,RiPP,fungal-RiPP


### Describe

In [9]:
# nucleotide lengths in kb
bgc_metadata['length_nt'] = bgc_metadata['length_nt']/1000

bgc_metadata_partial = bgc_metadata[bgc_metadata['on_contig_edge'] == 1] 
bgc_metadata_complete = bgc_metadata[bgc_metadata['on_contig_edge'] == 0]

In [10]:
# length-based statistics for different classes for complete BGCs
(bgc_metadata_complete[['chem_class_name', 'length_nt']].groupby("chem_class_name")).describe()

length_nt                                                \
                    count       mean         std    min     25%     50%   
chem_class_name                                                           
Alkaloid             49.0  57.676816  142.963593  5.918  20.179  26.086   
NRP              245618.0  56.018380   19.725351  2.788  44.020  52.262   
Other            204790.0  28.639913   19.440663  0.726  14.993  20.666   
Polyketide       157926.0  58.006659   22.525407  1.819  44.170  51.548   
RiPP             219547.0  19.702688   13.177695  0.204  10.807  17.442   
Saccharide         1173.0  50.452341   38.507196  2.815  21.236  41.029   
Terpene           97296.0  26.406301   13.713759  1.008  20.864  21.167   

                                    
                      75%      max  
chem_class_name                     
Alkaloid         40.73800  935.096  
NRP              59.17575  390.736  
Other            41.23000  409.731  
Polyketide       59.48600  390.736  
RiPP             26.27000  329.842  
Saccharide       67.31500  279.790  
Terpene          22.93625  377.016

In [11]:
# length-based statistics for different classes for partial BGCs
(bgc_metadata_partial[['chem_class_name', 'length_nt']].groupby("chem_class_name")).describe()

length_nt                                               \
                    count       mean        std    min     25%     50%   
chem_class_name                                                          
NRP              189463.0  35.713063  19.938403  3.187  22.755  33.757   
Other             80052.0  23.473785  17.753812  5.000  11.281  18.224   
Polyketide       102264.0  37.193082  21.554318  5.000  22.263  34.191   
RiPP              88942.0  13.016109  10.427010  3.680   7.279   9.141   
Saccharide          356.0  43.934896  32.985869  5.387  19.101  34.934   
Terpene           28433.0  19.054678  12.902904  5.002  12.677  15.737   

                                    
                      75%      max  
chem_class_name                     
NRP              45.78200  331.109  
Other            28.68900  293.812  
Polyketide       46.94175  331.109  
RiPP             17.12075  331.109  
Saccharide       59.38125  249.884  
Terpene          19.43400  242.587